## Libraries + API Keys + Parameters

In [ ]:
!pip install 'crewai[tools]' -q

In [ ]:
%cd /content/drive/MyDrive/AI Agents/CrewAI/Job Interview Prep Part 2

/content/drive/MyDrive/AI Agents/CrewAI/Job Interview Prep Part 2


In [ ]:
# Retrieve the API keys securely from Google Colab's user data
from google.colab import userdata
openai_api_key = userdata.get('openai_api')
serper_api_key = userdata.get('serper_api')

In [ ]:
import os
# Set the API keys as environment variables for accessibility
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [ ]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [ ]:
# Define the LLM
llm = ChatOpenAI(model_name='gpt-4o', temperature=0.5)

# Get the data

In [ ]:
# We need to know the Name, company, job position, job description
interviewer = input("Enter the name of the interviewer (if unknown write NONE): ")
company = input("Enter the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

Enter the name of the interviewer (if unknown write NONE): Nicole Malucha
Enter the company: Beat81
Enter the job position: (Senior) Supply Chain & Operations Manager
Enter the job description: ABOUT THE ROLE You will play a pivotal role in supporting our expansion team by ensuring equipment readiness for our seamless and timely launches of new BEAT studios. Additionally, you will oversee the operational excellence of our active gyms by ensuring all equipment and facilities are maintained to the highest standard, maximizing uptime and customer satisfaction.  This role demands exceptional organizational and analytical skills and a solutions-oriented mindset. Elevate BEAT81 by creating, standardizing, and maintaining procurement, inventory, and maintenance processes cross-functionally, driven by KPIs.    YOUR RESPONSIBILITIES   Procurement: Ensure timely and cost efficient procurement and delivery of BEAT branded fitness equipment, both internationally and nationally, to meet quantity an

# Research Crew

## Research Agent

In [ ]:
# Research Agent
research_agent = Agent(
    role = 'Research Agent',
    goal = f"Conduct in-depth research to serve as a basis for interview prep",
    backstory = """
    As a Research Specialist, your mission is to uncover detailed and relevant information about the interviewer and company.
        This includes their professional backgrounds, recent projects, social media activity, public speaking engagements, and any insights that can help tailor the interview discussion.
        The ultimate goal is to equip the candidate with backgrounds of the company and the interviewer.
        """,
    llm = llm,
    tools = [ScrapeWebsiteTool(), SerperDevTool()]
)

## Research Tasks

In [ ]:
# Research Task
research_company_task = Task(
    description = f"""
    Research about {company}  their position in the sector, and recent industry trends.
    Focus on news and developments from the last year, as the current
        year is 2024.""",
    expected_output=f"A comprehensive report in Markdown with bullet points about {company}",
    agent = research_agent
)

In [ ]:
# Research Task
research_person_task = Task(
    description = f"Research about {interviewer} with a focus on their professional background, recent projects, social media activity, and any insights that can help tailor the interview discussion.",
    expected_output=f"A comprehensive report in Markdown with bullet points about {interviewer}",
    agent = research_agent)

## Prepare interview

In [ ]:
# Research Agent
coach_agent = Agent(
    role = 'coach Agent',
    goal = f"Help prepare the user for job interview at {company} for {job_position}",
    backstory = """
    As an experienced interview coach, your task is to help prepare the user for the interview.
    The user should be asked questions, and be given advice on how to improve the answers.
    """,
    llm = llm,
)

In [ ]:
# Prepare list of 8 questions
define_questions_task = Task(
    description = f""" Define a list of questions based on the {company}, {interviewer} and {job_description} for the job {job_position}
    """,
    expected_output=f"A list of 8 questions in Markdown for culture fit and position fit",
    context = [research_company_task, research_person_task],
    agent = coach_agent
)

## Assemble research crew

In [ ]:
crew = Crew(
    agents=[research_agent, coach_agent],  # List of agents involved
    tasks=[research_company_task, research_person_task, define_questions_task],  # List of tasks to execute
    verbose=True,
    process = Process.sequential,
)
result = crew.kickoff({"topic": f"draft the question for the interview"})  # Start the task execution

# Agent: Research Agent
## Task: 
    Research about Beat81  their position in the sector, and recent industry trends.
    Focus on news and developments from the last year, as the current
        year is 2024.


# Agent: Research Agent
## Thought: Thought: To gather comprehensive information about Beat81, I will start by searching the internet for recent news and developments related to the company from the past year. This will help me understand their position in the sector and any relevant industry trends.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Beat81 company news 2024\"}"
## Tool Output: 
{'searchParameters': {'q': 'Beat81 company news 2024', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Home | BEAT81 | HIIT Functional Training & Group Fitness', 'link': 'https://www.beat81.com/', 'snippet': 'Smarter workouts to see results faster. Burn up to 1000 calories in just 45 minutes. Book a free workout. Reach your goals with Vari

## Exporting

In [ ]:
result.raw

"```markdown\n1. **Culture Fit:**\n   - Can you describe a time when you had to adapt to a fast-paced and dynamic work environment, similar to a startup setting like Beat81?\n   - How do you approach fostering a culture of openness, trust, and care within your team, as emphasized in Beat81's working environment?\n   - Beat81 values diversity and inclusion. How have you contributed to creating an inclusive environment in your previous roles?\n\n2. **Position Fit:**\n   - What experience do you have in developing and implementing procurement and inventory management processes, particularly in a company with physical operations like Beat81?\n   - Can you provide an example of a successful project where you optimized supply chain processes to enhance efficiency and scalability?\n   - Describe a situation where you had to use data-driven performance management to drive continuous improvement in supply chain operations.\n   - How do you ensure effective cross-functional collaboration to achi

In [ ]:
display(Markdown(result.raw))

```markdown
1. **Culture Fit:**
   - Can you describe a time when you had to adapt to a fast-paced and dynamic work environment, similar to a startup setting like Beat81?
   - How do you approach fostering a culture of openness, trust, and care within your team, as emphasized in Beat81's working environment?
   - Beat81 values diversity and inclusion. How have you contributed to creating an inclusive environment in your previous roles?

2. **Position Fit:**
   - What experience do you have in developing and implementing procurement and inventory management processes, particularly in a company with physical operations like Beat81?
   - Can you provide an example of a successful project where you optimized supply chain processes to enhance efficiency and scalability?
   - Describe a situation where you had to use data-driven performance management to drive continuous improvement in supply chain operations.
   - How do you ensure effective cross-functional collaboration to achieve business objectives, especially in a role that demands coordination with various departments?
   - What strategies would you employ to maintain the operational excellence of Beat81's gyms, ensuring equipment readiness and minimizing downtime?
```

These questions are designed to assess the candidate's ability to integrate into Beat81's culture and effectively manage the supply chain and operations tasks specific to the role.

In [ ]:
result.tasks_output[2].raw

"```markdown\n1. **Culture Fit:**\n   - Can you describe a time when you had to adapt to a fast-paced and dynamic work environment, similar to a startup setting like Beat81?\n   - How do you approach fostering a culture of openness, trust, and care within your team, as emphasized in Beat81's working environment?\n   - Beat81 values diversity and inclusion. How have you contributed to creating an inclusive environment in your previous roles?\n\n2. **Position Fit:**\n   - What experience do you have in developing and implementing procurement and inventory management processes, particularly in a company with physical operations like Beat81?\n   - Can you provide an example of a successful project where you optimized supply chain processes to enhance efficiency and scalability?\n   - Describe a situation where you had to use data-driven performance management to drive continuous improvement in supply chain operations.\n   - How do you ensure effective cross-functional collaboration to achi

In [ ]:
# Define the file name
file_name = f'{company}_report.txt'

# Open the file in write mode and write the content
with open(file_name, "w", encoding="utf-8") as file:
    file.write(result.tasks_output[0].raw)

print(f"Data successfully exported to {file_name}")


Data successfully exported to Beat81_report.txt


In [ ]:
# Define the file name
file_name = f'{interviewer}_report.txt'

# Open the file in write mode and write the content
with open(file_name, "w", encoding="utf-8") as file:
    file.write(result.tasks_output[1].raw)

print(f"Data successfully exported to {file_name}")


Data successfully exported to Nicole Malucha_report.txt


In [ ]:
# Define the file name
file_name = f'{job_position}_interview_questions.txt'

# Open the file in write mode and write the content
with open(file_name, "w", encoding="utf-8") as file:
    file.write(result.tasks_output[2].raw)

print(f"Data successfully exported to {file_name}")


Data successfully exported to (Senior) Supply Chain & Operations Manager_interview_questions.txt


# Interview Prep

In [ ]:
# Define the file name
file_name = f'{job_position}_interview_questions.txt'

# Open the file in read mode and read the content
with open(file_name, "r", encoding="utf-8") as file:
    questions_data = file.read()

In [ ]:
# Research Agent
interviewer_agent = Agent(
    role = 'Interview Agent',
    goal = f"Help prepare the user for job interview at {company} for {job_position}",
    backstory = f"""
    As an experienced interviewer like {interviewer}, your task is to help prepare the user for the interview.
    The user should be asked questions, and be given advice on how to improve the answers.
    """,
    llm = llm,
)

In [ ]:
# Prepare list of 8 questions
asking_task = Task(
    description = f""" ask one random question from {questions_data} from the defined list of questions in Markdown for culture fit and position fit in the knowledge source
    """,
    expected_output=f"A new question to the user",
    agent = interviewer_agent,
    human_input = True
)

In [ ]:
# Grading Questions
feedback_task = Task(
    description = f""" a list of positives and negatives from the user's answer to the question
    """,
    expected_output=f"feedback of the user's input to the question",
    context = [asking_task],
    agent = coach_agent,
    human_input = True
)

## Assemble the Crew

In [ ]:
crew = Crew(
    agents=[coach_agent, interviewer_agent],  # List of agents involved
    tasks=[asking_task, feedback_task],  # List of tasks to execute
    verbose=True,
    rocess = Process.hierarchical,
    manager_agent = coach_agent,
    cache = True,
    memory = True,
    planning = True
)
result = crew.kickoff({"topic": "Ask me a question from the knowledge_sources"})  # Start the task execution

 
[2025-01-16 11:04:21][INFO]: Planning the crew execution
# Agent: Interview Agent
## Task:  ask one random question from ```markdown
1. **Culture Fit:**
   - Can you describe a time when you had to adapt to a fast-paced and dynamic work environment, similar to a startup setting like Beat81?
   - How do you approach fostering a culture of openness, trust, and care within your team, as emphasized in Beat81's working environment?
   - Beat81 values diversity and inclusion. How have you contributed to creating an inclusive environment in your previous roles?

2. **Position Fit:**
   - What experience do you have in developing and implementing procurement and inventory management processes, particularly in a company with physical operations like Beat81?
   - Can you provide an example of a successful project where you optimized supply chain processes to enhance efficiency and scalability?
   - Describe a situation where you had to use data-driven performance management to drive continuous

KeyboardInterrupt: Interrupted by user